In [2]:
import requests
import datetime
import bs4
import time
import sqlite3
from sqlite3 import Error

In [3]:
def dbDsda():
    return r"dsda.db"


def create_connection(db_file):
    '''
    create a database connection to a SQLite database
    specified by db_file
    :param db_file : database file
    :return: Connection Object or None
    '''
    conn=None
    try:
        conn=sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
def create_table(conn, create_table_sql):
    '''
    create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    '''
    try:
        c=conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_create_dsdadb(conn, stationName):
    sql_create_table="""CREATE TABLE IF NOT EXISTS {}(currentdate text UNIQUE, data text)""".format(stationName)
    
    #create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_table)
        print("Success")
    else:
        print("Error! cannot create the database conenction.")
        
def insert_dsdadb(conn, table, datevalues, datavalues):
    '''
    Insert new datevalues to dsda table
    :param conn:
    :param datevalues: text in sqlite datetime format
    '''
    sql='''insert into {}(currentdate, data) values("{}","{}")'''.format(table,datevalues,datavalues)
    cur=conn.cursor()
    try:
        cur.execute(sql,)
        conn.commit()
        return "Data insert success"
    except Error:
        return "Data insert Error - Already inserted"
        

def currentStartEndDate(stationCode, yearList, totalData, currentLoop):
    '''
    Return a pair of tuple data containing start and end date for data crawling.
    Also, by passing stationCode (unique), this function will store current loop performed on the disk
    '''
    
    
    #print(currentLoop)

    # from 2000 to 2021
    #yearList=['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']

    # month : 01 - 12
    monthList=['01','02','03','04','05','06','07','08','09','10','11','12']

    # daylist : 
    startDay='01'
    endDayListKabisat=['31','29','31','30','31','30','31','31','30','31','30','31']
    endDayListN=['31','28','31','30','31','30','31','31','30','31','30','31']


    #currentLoop=0
    #totalData=len(yearList)*len(monthList)
    monthIndex=currentLoop%12
    yearIndex=currentLoop//12
    if monthIndex == 12:
        monthIndex=0
        yearIndex=yearIndex+1   


    if currentLoop < totalData:
        '''
        Performing loop in current Year list to a 1-month-basis for crawling purpose
        '''
        # START DATE
        # assign data section
        year=yearList[yearIndex]
        month=monthList[monthIndex]

        # get day data based on month and 'kabisat' year identification
        checkRemainder=int(year)%4
        if checkRemainder != 0:
            day=endDayListN[monthIndex]
        else:
            day=endDayListKabisat[monthIndex]

        # increment section
        if monthIndex < 11:
            monthIndex+=1
        else:
            monthIndex=0
            yearIndex+=1
        currentLoop+=1
        
        # first tuple index is for identification, 0 if the loop is ongoing, 1 if we are already on the end of the loop
        return ('0','{}'.format(currentLoop),'{}/{}/{}'.format(startDay,month,year), '{}/{}/{}'.format(day,month,year))
        
    else: # else we already on the end of the loop
        return ('1','0','0','0') 
        
def dsdaRequestData(currentStation, yearList, totalData, currentLoop):
    '''
    Return dictionary of data from dsda website for current loop and station
    '''
    check, currentLoopStatus, startDate, endDate = currentStartEndDate(currentStation, yearList, totalData, currentLoop)
    print('{}/{} : {}-{}'.format(currentLoop+1, totalData, startDate, endDate))
    if check=='0':
        url='http://poskobanjirdsda.jakarta.go.id/Pages/GenerateDataTinggiAir.aspx?IdPintuAir={}&StartDate={}&EndDate={}'.format(currentStation,startDate,endDate)
        # test URL down below (worked!)
        #url='http://poskobanjirdsda.jakarta.go.id/Pages/GenerateDataTinggiAir.aspx?IdPintuAir=122&StartDate=02/03/2021&EndDate=03/03/2021'
        
        try:
            response=requests.get(url)

                
            # start processing response content
            # beautify text got from the response content
            soup = bs4.BeautifulSoup(response.content)
            textOnly=soup.text
            # split with "|"
            twoKindData=textOnly.split("|")
            mainData=twoKindData[0]
            # split again with ';'
            mainDataLists=mainData.split(';')
            # delete last item in list, because contained empty data
            mainDataLists.pop(len(mainDataLists)-1)
            dataDictionary={}
            for data in mainDataLists:
                # last split with ','
                dateText, dataText=data.split(',')
                dateFormatted='{}:{}'.format(dateText[:13], dateText[14:16])
                dataDictionary[dateFormatted]=dataText
                
            # update current loop on file
            with open('dcl-{}.cache'.format(currentStation),'w') as dcl:
                dcl.write(str(currentLoopStatus))

            return dataDictionary

        except:
            # update current loop on file with last failed fetch batch
            with open('dcl-{}.cache'.format(currentStation),'w') as dcl:
                dcl.write(str(currentLoopStatus-1))

            return 0
    
        
    else:
        # update current loop on file with 0 again (meaning that we are ready start again from zero)
        with open('dcl-{}.cache'.format(stationCode),'w') as dcl:
            dcl.write('0')
        
    

    
def executeBatchPastDsdaData(currentStation, stationName, yearList):
    '''
    yearList=['2000','2001','2002','2003','2004','2005',
    '2006','2007','2008','2009','2010','2011','2012','2013',
    '2014','2015','2016','2017','2018','2019','2020','2021']
    
    '''
    # check stored current loop
    try:
        with open('dcl-{}.cache'.format(currentStation),'r') as dcl:
            fetchCurrentLoop=dcl.read()
            currentLoop=int(fetchCurrentLoop)
    except:
        currentLoop=0
        with open('dcl-{}.cache'.format(currentStation),'w') as dcl:
            dcl.write(str(currentLoop))
    
    totalData=len(yearList)*12
    
    while currentLoop < totalData:
        print('Start fetching data')
        start_time=time.time()
        conn=create_connection(dbDsda())        
        datas=dsdaRequestData(currentStation, yearList, totalData, currentLoop)
        
        # if error occured, continue on the same batch of data after sleep
        if datas == 0:
            print("Failed to reach server. Trying again in 30 seconds")
            time.sleep(30)
            continue
        
        # refetching current station progress
        try:
            with open('dcl-{}.cache'.format(currentStation),'r') as dcl:
                fetchCurrentLoop=dcl.read()
                currentLoop=int(fetchCurrentLoop)
        except:
            currentLoop=0
            with open('dcl-{}.cache'.format(currentStation),'w') as dcl:
                dcl.write(str(currentLoop))
        
        
        
        execute_create_dsdadb(conn, stationName)
        for data in datas:
            insertData=insert_dsdadb(conn, stationName, data, datas[data])
            
        # performing sleep for every fetch loop to prevent abusive behaviour
        finish_time=time.time()
        print('Finish data processing in {} secs, sleep for 10s'.format(finish_time-start_time))
        time.sleep(10)
        
    
        
    

In [4]:
'''
crawledStation={'114':'krukut', #2014
               '144':'cideng', #2015
               '143':'karet', #2015
               '140':'marinaancol', #2015
               '117':'pasarikan', #2014
               '141':'pluit', #2015
               '108':'pesanggrahan', #2013
               '106':'angkehulu',
               '135':'sunterhulu',
               '126':'pulogadung',
               '142':'yossudarso1',
               '103':'cipinanghulu',
               '145':'kalijodo',
               '148':'istiqlal',
               '147':'jembatanmerah',
               '146':'flusingancol',
               '149':'hek'}
               
crawledStation={'142':'yossudarso1',
               '103':'cipinanghulu',
               '145':'kalijodo',
               '148':'istiqlal',
               '147':'jembatanmerah',
               '146':'flusingancol',
               '149':'hek'}
'''
crawledStation={'146':'flusingancol',
               '149':'hek'}

yearList=['2013','2014','2015','2016','2017','2018','2019','2020','2021']
for station in crawledStation:
    print('Crawling {} started'.format(crawledStation[station]))
    executeBatchPastDsdaData(station, crawledStation[station], yearList)
    print('Finish crawling {}!'.format(crawledStation[station]))

Crawling flusingancol started
Start fetching data
1/108 : 01/01/2013-31/01/2013
Success
Finish data processing in 0.2082533836364746 secs, sleep for 10s
Start fetching data
2/108 : 01/02/2013-28/02/2013
Success
Finish data processing in 0.17954683303833008 secs, sleep for 10s
Start fetching data
3/108 : 01/03/2013-31/03/2013
Success
Finish data processing in 0.22221946716308594 secs, sleep for 10s
Start fetching data
4/108 : 01/04/2013-30/04/2013
Success
Finish data processing in 0.1753554344177246 secs, sleep for 10s
Start fetching data
5/108 : 01/05/2013-31/05/2013
Success
Finish data processing in 0.1839609146118164 secs, sleep for 10s
Start fetching data
6/108 : 01/06/2013-30/06/2013
Success
Finish data processing in 0.21274256706237793 secs, sleep for 10s
Start fetching data
7/108 : 01/07/2013-31/07/2013
Success
Finish data processing in 1.948564052581787 secs, sleep for 10s
Start fetching data
8/108 : 01/08/2013-31/08/2013
Success
Finish data processing in 0.17862343788146973 sec

Start fetching data
67/108 : 01/07/2018-31/07/2018
Success
Finish data processing in 0.823373556137085 secs, sleep for 10s
Start fetching data
68/108 : 01/08/2018-31/08/2018
Success
Finish data processing in 0.6719129085540771 secs, sleep for 10s
Start fetching data
69/108 : 01/09/2018-30/09/2018
Success
Finish data processing in 0.1989915370941162 secs, sleep for 10s
Start fetching data
70/108 : 01/10/2018-31/10/2018
Success
Finish data processing in 14.685071229934692 secs, sleep for 10s
Start fetching data
71/108 : 01/11/2018-30/11/2018
Success
Finish data processing in 8.67942214012146 secs, sleep for 10s
Start fetching data
72/108 : 01/12/2018-31/12/2018
Success
Finish data processing in 22.9742431640625 secs, sleep for 10s
Start fetching data
73/108 : 01/01/2019-31/01/2019
Success
Finish data processing in 27.36776852607727 secs, sleep for 10s
Start fetching data
74/108 : 01/02/2019-28/02/2019
Success
Finish data processing in 23.19878077507019 secs, sleep for 10s
Start fetching 

Start fetching data
25/108 : 01/01/2015-31/01/2015
Success
Finish data processing in 1.8750147819519043 secs, sleep for 10s
Start fetching data
26/108 : 01/02/2015-28/02/2015
Success
Finish data processing in 2.8560104370117188 secs, sleep for 10s
Start fetching data
27/108 : 01/03/2015-31/03/2015
Success
Finish data processing in 0.19947338104248047 secs, sleep for 10s
Start fetching data
28/108 : 01/04/2015-30/04/2015
Success
Finish data processing in 0.19212722778320312 secs, sleep for 10s
Start fetching data
29/108 : 01/05/2015-31/05/2015
Success
Finish data processing in 0.16776347160339355 secs, sleep for 10s
Start fetching data
30/108 : 01/06/2015-30/06/2015
Success
Finish data processing in 0.2705221176147461 secs, sleep for 10s
Start fetching data
31/108 : 01/07/2015-31/07/2015
Success
Finish data processing in 0.19663333892822266 secs, sleep for 10s
Start fetching data
32/108 : 01/08/2015-31/08/2015
Success
Finish data processing in 0.18274307250976562 secs, sleep for 10s
Sta

Start fetching data
91/108 : 01/07/2020-31/07/2020
Success
Finish data processing in 21.805479049682617 secs, sleep for 10s
Start fetching data
92/108 : 01/08/2020-31/08/2020
Success
Finish data processing in 25.130834817886353 secs, sleep for 10s
Start fetching data
93/108 : 01/09/2020-30/09/2020
Success
Finish data processing in 14.586358547210693 secs, sleep for 10s
Start fetching data
94/108 : 01/10/2020-31/10/2020
Success
Finish data processing in 2.7388415336608887 secs, sleep for 10s
Start fetching data
95/108 : 01/11/2020-30/11/2020
Success
Finish data processing in 20.410541534423828 secs, sleep for 10s
Start fetching data
96/108 : 01/12/2020-31/12/2020
Success
Finish data processing in 21.695977210998535 secs, sleep for 10s
Start fetching data
97/108 : 01/01/2021-31/01/2021
Success
Finish data processing in 23.708906173706055 secs, sleep for 10s
Start fetching data
98/108 : 01/02/2021-28/02/2021
Success
Finish data processing in 19.20227885246277 secs, sleep for 10s
Start fet

In [3]:
stationData=[{ 'value': '115', 'text': 'Bendung. Cibalok - Gadog' }, { 'value': '116', 'text': 'Bendung. Katulampa (Hulu)' }, { 'value': '118', 'text': 'PS. Depok' }, { 'value': '122', 'text': 'PA. Manggarai' }, { 'value': '114', 'text': 'PS. Krukut Hulu' }, { 'value': '144', 'text': 'Pompa Cideng' }, { 'value': '143', 'text': 'P.A. Karet' }, { 'value': '140', 'text': 'P.A. Marina Ancol ' }, { 'value': '117', 'text': 'Pompa Pasar Ikan' }, { 'value': '141', 'text': 'Pompa. Pluit' }, { 'value': '108', 'text': 'PS. Pesanggrahan' }, { 'value': '106', 'text': 'PS. Angke Hulu' }, { 'value': '135', 'text': 'PS. Sunter Hulu' }, { 'value': '126', 'text': 'PA. Pulo Gadung' }, { 'value': '142', 'text': 'Pompa Yos Sudarso 1' }, { 'value': '103', 'text': 'PS. Cipinang Hulu' }, { 'value': '145', 'text': 'Pompa Kali Duri (Kalijodo)' }, { 'value': '148', 'text': 'P.A. Istiqlal  ' }, { 'value': '147', 'text': 'P.A. Jembatan Merah' }, { 'value': '146', 'text': 'P.A. Flusing Ancol' }, { 'value': '149', 'text': 'P.A. Hek' }]

In [4]:
[{ 'value': '115', 'text': 'Bendung. Cibalok - Gadog' }, { 'value': '116', 'text': 'Bendung. Katulampa (Hulu)' }, { 'value': '118', 'text': 'PS. Depok' }, { 'value': '122', 'text': 'PA. Manggarai' }, { 'value': '114', 'text': 'PS. Krukut Hulu' }, { 'value': '144', 'text': 'Pompa Cideng' }, { 'value': '143', 'text': 'P.A. Karet' }, { 'value': '140', 'text': 'P.A. Marina Ancol ' }, { 'value': '117', 'text': 'Pompa Pasar Ikan' }, { 'value': '141', 'text': 'Pompa. Pluit' }, { 'value': '108', 'text': 'PS. Pesanggrahan' }, { 'value': '106', 'text': 'PS. Angke Hulu' }, { 'value': '135', 'text': 'PS. Sunter Hulu' }, { 'value': '126', 'text': 'PA. Pulo Gadung' }, { 'value': '142', 'text': 'Pompa Yos Sudarso 1' }, { 'value': '103', 'text': 'PS. Cipinang Hulu' }, { 'value': '145', 'text': 'Pompa Kali Duri (Kalijodo)' }, { 'value': '148', 'text': 'P.A. Istiqlal  ' }, { 'value': '147', 'text': 'P.A. Jembatan Merah' }, { 'value': '146', 'text': 'P.A. Flusing Ancol' }, { 'value': '149', 'text': 'P.A. Hek' }]

[{'value': '115', 'text': 'Bendung. Cibalok - Gadog'},
 {'value': '116', 'text': 'Bendung. Katulampa (Hulu)'},
 {'value': '118', 'text': 'PS. Depok'},
 {'value': '122', 'text': 'PA. Manggarai'},
 {'value': '114', 'text': 'PS. Krukut Hulu'},
 {'value': '144', 'text': 'Pompa Cideng'},
 {'value': '143', 'text': 'P.A. Karet'},
 {'value': '140', 'text': 'P.A. Marina Ancol '},
 {'value': '117', 'text': 'Pompa Pasar Ikan'},
 {'value': '141', 'text': 'Pompa. Pluit'},
 {'value': '108', 'text': 'PS. Pesanggrahan'},
 {'value': '106', 'text': 'PS. Angke Hulu'},
 {'value': '135', 'text': 'PS. Sunter Hulu'},
 {'value': '126', 'text': 'PA. Pulo Gadung'},
 {'value': '142', 'text': 'Pompa Yos Sudarso 1'},
 {'value': '103', 'text': 'PS. Cipinang Hulu'},
 {'value': '145', 'text': 'Pompa Kali Duri (Kalijodo)'},
 {'value': '148', 'text': 'P.A. Istiqlal  '},
 {'value': '147', 'text': 'P.A. Jembatan Merah'},
 {'value': '146', 'text': 'P.A. Flusing Ancol'},
 {'value': '149', 'text': 'P.A. Hek'}]

In [5]:
crawledStation={'114':'krukut',
               '144':'cideng',
               '143':'karet',
               '140':'marinaancol',
               '117':'pasarikan',
               '141':'pluit',
               '108':'pesanggrahan',
               '106':'angkehulu',
               '135':'sunterhulu',
               '126':'pulogadung',
               '142':'yossudarso1',
               '103':'cipinanghulu',
               '145':'kalijodo',
               '148':'istiqlal',
               '147':'jembatanmerah',
               '146':'flusingancol',
               '149':'hek'}

In [3]:
#sdrin=['https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-12@%@0@%@0@%@0@%@1201859@%@1004117@%@32656@%@9398@%@7759@%@380@%@29978@%@26218@%@783@%@26957@%@19669@%@601@%@5823@%@4959@%@96@%@4694@%@4437@%@145@%@24812@%@18053@%@575@%@310039@%@284380@%@4810@%@4570@%@4161@%@123@%@4991@%@3801@%@77@%@172384@%@143374@%@2107@%@140196@%@90339@%@5747@%@121299@%@107344@%@8467@%@4233@%@3842@%@32@%@48528@%@38752@%@1134@%@12628@%@11236@%@333@%@19452@%@17136@%@680@%@8333@%@5496@%@127@%@8439@%@7891@%@209@%@8068@%@6312@%@335@%@6785@%@3662@%@187@%@15027@%@12684@%@728@%@28010@%@26519@%@630@%@14375@%@10715@%@485@%@22585@%@19429@%@777@%@9816@%@8822@%@188@%@51745@%@46918@%@781@%@9132@%@6444@%@222@%@11293@%@8844@%@594@%@30027@%@28251@%@720@%@3820@%@3043@%@110@%@6640@%@5767@%@101@%@6969@%@6334@%@112@%@16046@%@8842@%@171@%@4767@%@2684@%@89@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-13@%@0@%@0@%@0@%@1210703@%@1016036@%@32936@%@9399@%@7762@%@382@%@30134@%@26444@%@797@%@27071@%@19704@%@601@%@5956@%@5078@%@97@%@4703@%@4440@%@145@%@24925@%@18423@%@582@%@313057@%@289126@%@4863@%@4574@%@4188@%@123@%@5024@%@3804@%@77@%@174121@%@145857@%@2109@%@140959@%@90896@%@5856@%@121859@%@108010@%@8512@%@4243@%@3865@%@32@%@48842@%@39243@%@1144@%@12729@%@11336@%@335@%@19596@%@17287@%@683@%@8472@%@5546@%@128@%@8459@%@7898@%@209@%@8085@%@6370@%@335@%@7020@%@4013@%@190@%@15084@%@12758@%@729@%@28091@%@26597@%@630@%@14459@%@10768@%@486@%@22739@%@19534@%@777@%@9824@%@8873@%@188@%@52126@%@47385@%@792@%@9209@%@6586@%@227@%@11368@%@9057@%@599@%@30074@%@28355@%@722@%@3847@%@3048@%@110@%@6714@%@5780@%@103@%@6977@%@6370@%@112@%@16101@%@8849@%@171@%@4862@%@2786@%@90@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-14@%@0@%@0@%@0@%@1217468@%@1025273@%@33183@%@9403@%@7762@%@382@%@30273@%@26671@%@805@%@27145@%@19716@%@602@%@6086@%@5152@%@97@%@4721@%@4450@%@145@%@25033@%@18670@%@590@%@315553@%@293463@%@4906@%@4593@%@4199@%@123@%@5058@%@3826@%@77@%@175003@%@147551@%@2113@%@141437@%@90944@%@5924@%@122375@%@108676@%@8562@%@4262@%@3881@%@32@%@49172@%@39743@%@1161@%@12765@%@11440@%@335@%@19693@%@17350@%@687@%@8551@%@5662@%@128@%@8492@%@7915@%@209@%@8098@%@6445@%@336@%@7272@%@4170@%@196@%@15150@%@12882@%@731@%@28168@%@26325@%@630@%@14515@%@10847@%@488@%@22865@%@19641@%@781@%@9839@%@8880@%@190@%@52480@%@47646@%@805@%@9266@%@6693@%@230@%@11457@%@9154@%@600@%@30141@%@28441@%@723@%@3869@%@3058@%@110@%@6747@%@5876@%@103@%@7001@%@6385@%@119@%@16116@%@8860@%@171@%@4869@%@2899@%@92@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-15@%@0@%@0@%@0@%@1223930@%@1032065@%@33367@%@9403@%@7762@%@382@%@30547@%@26999@%@814@%@27324@%@19796@%@602@%@6224@%@5274@%@97@%@4726@%@4465@%@145@%@25168@%@18810@%@595@%@317432@%@295545@%@4961@%@4612@%@4209@%@124@%@5067@%@3842@%@77@%@175950@%@148816@%@2118@%@142318@%@91170@%@5950@%@122807@%@109400@%@8613@%@4262@%@3881@%@32@%@49534@%@40227@%@1168@%@12806@%@11486@%@338@%@19839@%@17426@%@691@%@8736@%@5696@%@131@%@8500@%@7916@%@209@%@8101@%@6456@%@336@%@7387@%@4393@%@196@%@15201@%@12990@%@732@%@28182@%@26342@%@634@%@14612@%@11014@%@488@%@22999@%@19758@%@786@%@9847@%@8888@%@190@%@52640@%@47862@%@805@%@9315@%@6740@%@232@%@11525@%@9248@%@601@%@30197@%@28492@%@725@%@3878@%@3111@%@110@%@6747@%@5876@%@103@%@7028@%@6392@%@119@%@16133@%@8872@%@171@%@4883@%@2911@%@92@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-16@%@0@%@0@%@0@%@1233959@%@1039674@%@33596@%@9403@%@7762@%@382@%@30821@%@27313@%@823@%@27675@%@19892@%@604@%@6278@%@5382@%@99@%@4743@%@4475@%@146@%@25335@%@19096@%@599@%@319293@%@297641@%@5010@%@4634@%@4212@%@124@%@5091@%@3849@%@77@%@179982@%@150916@%@2132@%@143419@%@91420@%@6016@%@123280@%@109978@%@8668@%@4299@%@3913@%@32@%@49856@%@40451@%@1171@%@12856@%@11516@%@338@%@19978@%@17562@%@694@%@8811@%@5757@%@133@%@8514@%@7923@%@209@%@8158@%@6585@%@336@%@7517@%@4685@%@199@%@15253@%@13107@%@733@%@28236@%@26405@%@636@%@14629@%@11057@%@488@%@23108@%@19862@%@789@%@9858@%@8892@%@190@%@52959@%@48088@%@809@%@9346@%@6777@%@232@%@11603@%@9283@%@602@%@30277@%@28562@%@729@%@3886@%@3111@%@110@%@6769@%@5982@%@103@%@7036@%@6392@%@119@%@16135@%@8887@%@171@%@4921@%@2941@%@93@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-17@%@0@%@0@%@0@%@1243646@%@1047676@%@33788@%@9403@%@7762@%@382@%@31195@%@27548@%@835@%@28009@%@19935@%@604@%@6394@%@5439@%@101@%@4752@%@4485@%@146@%@25562@%@19340@%@609@%@320738@%@299740@%@5057@%@4638@%@4214@%@124@%@5113@%@3879@%@78@%@184106@%@153619@%@2135@%@144288@%@91620@%@6059@%@123860@%@110575@%@8702@%@4325@%@3934@%@32@%@50308@%@40865@%@1180@%@12888@%@11578@%@338@%@20129@%@17670@%@698@%@8871@%@5914@%@133@%@8527@%@7923@%@210@%@8219@%@6647@%@336@%@7620@%@4839@%@202@%@15308@%@13166@%@737@%@28307@%@26550@%@637@%@14633@%@11073@%@488@%@23236@%@19973@%@794@%@9872@%@8921@%@190@%@53075@%@48303@%@815@%@9390@%@6811@%@235@%@11678@%@9327@%@602@%@30299@%@28670@%@732@%@3908@%@3111@%@110@%@6774@%@5982@%@103@%@7084@%@6399@%@119@%@16143@%@8887@%@171@%@4994@%@2977@%@94@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-18@%@0@%@0@%@0@%@1252685@%@1058222@%@33969@%@9435@%@7767@%@382@%@31501@%@27896@%@843@%@28095@%@20209@%@605@%@6521@%@5572@%@102@%@4757@%@4495@%@146@%@25817@%@19458@%@619@%@321111@%@302435@%@5089@%@4648@%@4225@%@126@%@5168@%@3901@%@80@%@188526@%@156868@%@2144@%@144964@%@92009@%@6082@%@124419@%@111223@%@8755@%@4357@%@3968@%@32@%@50826@%@41357@%@1191@%@12974@%@11680@%@345@%@20249@%@17724@%@700@%@8991@%@6172@%@134@%@8552@%@8031@%@210@%@8257@%@6662@%@336@%@7844@%@5027@%@204@%@15352@%@13252@%@742@%@28419@%@26657@%@637@%@14660@%@11201@%@488@%@23335@%@20124@%@798@%@9900@%@8959@%@191@%@53348@%@48524@%@815@%@9483@%@6926@%@237@%@11753@%@9679@%@603@%@30408@%@28730@%@734@%@3912@%@3140@%@111@%@6793@%@6012@%@103@%@7109@%@6428@%@120@%@16176@%@8895@%@171@%@5025@%@3016@%@94@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-19@%@0@%@0@%@0@%@1263299@%@1069005@%@34152@%@9448@%@7776@%@382@%@31831@%@28340@%@856@%@28168@%@20553@%@605@%@6606@%@5737@%@103@%@4771@%@4510@%@146@%@26080@%@19717@%@624@%@323031@%@304799@%@5095@%@4695@%@4238@%@126@%@5178@%@3927@%@80@%@192373@%@159519@%@2169@%@146170@%@92458@%@6113@%@124991@%@111822@%@8798@%@4383@%@3991@%@32@%@51544@%@41868@%@1206@%@13073@%@11713@%@345@%@20429@%@18006@%@701@%@9051@%@6661@%@136@%@8563@%@8102@%@212@%@8315@%@6673@%@336@%@7977@%@5162@%@210@%@15402@%@13283@%@744@%@28509@%@26730@%@639@%@14716@%@11275@%@488@%@23462@%@20239@%@803@%@9917@%@9076@%@191@%@53569@%@49051@%@819@%@9529@%@7085@%@242@%@11824@%@9946@%@612@%@30477@%@28802@%@737@%@3913@%@3202@%@111@%@6820@%@6035@%@104@%@7123@%@6436@%@120@%@16319@%@8928@%@171@%@5042@%@3345@%@96@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-20@%@0@%@0@%@0@%@1271353@%@1078840@%@34316@%@9456@%@7794@%@382@%@32033@%@28809@%@863@%@28241@%@20877@%@607@%@6738@%@5836@%@104@%@4790@%@4525@%@146@%@26281@%@19846@%@627@%@325903@%@307685@%@5135@%@4701@%@4250@%@127@%@5198@%@3955@%@80@%@193348@%@161047@%@2174@%@146975@%@92914@%@6131@%@125592@%@112411@%@8834@%@4418@%@4007@%@33@%@52096@%@42679@%@1224@%@13168@%@11801@%@346@%@20612@%@18256@%@703@%@9071@%@6752@%@136@%@8580@%@8128@%@213@%@8354@%@6691@%@336@%@8153@%@5332@%@214@%@15490@%@13340@%@745@%@28590@%@26840@%@641@%@14772@%@11429@%@488@%@23561@%@20357@%@805@%@9932@%@9111@%@191@%@53771@%@49447@%@825@%@9574@%@7517@%@242@%@11922@%@10008@%@613@%@30610@%@28973@%@743@%@3932@%@3222@%@111@%@6832@%@6094@%@105@%@7151@%@6566@%@122@%@16437@%@8940@%@172@%@5071@%@3401@%@98@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-21@%@0@%@0@%@0@%@1278653@%@1087076@%@34489@%@9457@%@7794@%@382@%@32359@%@29084@%@871@%@28490@%@21240@%@609@%@6793@%@5890@%@105@%@4803@%@4535@%@147@%@26456@%@20101@%@635@%@328628@%@310366@%@5190@%@4719@%@4259@%@128@%@5219@%@3979@%@81@%@194369@%@162670@%@2178@%@147462@%@93142@%@6143@%@126033@%@112980@%@8868@%@4448@%@4028@%@33@%@52502@%@43297@%@1236@%@13316@%@11826@%@346@%@20774@%@18437@%@707@%@9082@%@6846@%@136@%@8604@%@8150@%@217@%@8404@%@6697@%@336@%@8254@%@5584@%@217@%@15542@%@13381@%@752@%@28661@%@26911@%@641@%@14814@%@11444@%@488@%@23658@%@20455@%@808@%@9954@%@9112@%@191@%@53951@%@49781@%@825@%@9597@%@7588@%@245@%@11995@%@10140@%@618@%@30807@%@29062@%@744@%@3933@%@3226@%@111@%@6835@%@6107@%@105@%@7172@%@6586@%@122@%@16477@%@8952@%@175@%@5085@%@3426@%@99@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-22@%@0@%@0@%@0@%@1288833@%@1096994@%@34691@%@9458@%@7799@%@382@%@32637@%@29412@%@877@%@28603@%@21463@%@609@%@6864@%@5969@%@105@%@4825@%@4540@%@147@%@26594@%@20328@%@642@%@331094@%@312671@%@5230@%@4721@%@4276@%@128@%@5239@%@3999@%@81@%@198181@%@165502@%@2194@%@148509@%@94267@%@6184@%@126467@%@113513@%@8913@%@4448@%@4028@%@33@%@52906@%@43800@%@1250@%@13380@%@11872@%@348@%@20881@%@18563@%@710@%@9177@%@6914@%@138@%@8626@%@8171@%@217@%@8479@%@6700@%@336@%@8391@%@5918@%@220@%@15587@%@13524@%@756@%@28690@%@26937@%@641@%@14837@%@11539@%@490@%@23755@%@20560@%@813@%@9982@%@9139@%@191@%@54209@%@49985@%@827@%@9659@%@7734@%@246@%@12087@%@10319@%@625@%@30858@%@29155@%@746@%@3948@%@3275@%@111@%@6879@%@6107@%@105@%@7203@%@6611@%@122@%@16568@%@8952@%@175@%@5091@%@3452@%@99@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-23@%@0@%@0@%@0@%@1298608@%@1104990@%@35014@%@9487@%@7806@%@382@%@32925@%@29721@%@887@%@28854@%@21622@%@610@%@6905@%@6012@%@106@%@4867@%@4550@%@147@%@26822@%@20567@%@652@%@331876@%@314485@%@5275@%@4731@%@4290@%@130@%@5265@%@4026@%@81@%@202515@%@167551@%@2217@%@149336@%@94916@%@6326@%@127013@%@114024@%@8958@%@4476@%@4071@%@33@%@53337@%@44341@%@1257@%@13455@%@11961@%@350@%@20982@%@18673@%@714@%@9370@%@6931@%@142@%@8642@%@8172@%@217@%@8512@%@6760@%@337@%@8586@%@6040@%@223@%@15647@%@13680@%@761@%@28750@%@26991@%@642@%@14848@%@11575@%@490@%@23894@%@20667@%@819@%@9989@%@9184@%@192@%@54715@%@50255@%@827@%@9749@%@7905@%@248@%@12181@%@10445@%@627@%@30974@%@29214@%@749@%@3958@%@3330@%@112@%@6888@%@6118@%@105@%@7235@%@6630@%@124@%@16644@%@8969@%@175@%@5180@%@3508@%@99@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-24@%@0@%@0@%@0@%@1306141@%@1112725@%@35254@%@9513@%@7815@%@382@%@33232@%@30013@%@894@%@28895@%@21627@%@610@%@7141@%@6078@%@108@%@4883@%@4560@%@147@%@27023@%@20820@%@658@%@332658@%@316928@%@5326@%@4733@%@4298@%@130@%@5321@%@4075@%@81@%@204706@%@168930@%@2224@%@150598@%@95485@%@6401@%@127593@%@114572@%@8998@%@4503@%@4086@%@33@%@53760@%@44882@%@1271@%@13517@%@12028@%@352@%@21115@%@18781@%@717@%@9378@%@7100@%@142@%@8657@%@8191@%@218@%@8532@%@6785@%@337@%@8681@%@6182@%@243@%@15704@%@13721@%@763@%@28837@%@27119@%@642@%@14863@%@11702@%@490@%@24016@%@20767@%@820@%@9999@%@9201@%@192@%@55103@%@50549@%@830@%@9814@%@7947@%@249@%@12249@%@10538@%@630@%@31084@%@29269@%@751@%@3970@%@3330@%@112@%@6902@%@6137@%@105@%@7287@%@6664@%@124@%@16684@%@8988@%@175@%@5190@%@3557@%@99@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-25@%@0@%@0@%@0@%@1314634@%@1121411@%@35518@%@9523@%@7815@%@383@%@33495@%@30305@%@901@%@29158@%@21719@%@610@%@7186@%@6364@%@109@%@4894@%@4560@%@147@%@27251@%@21075@%@661@%@334239@%@319248@%@5366@%@4755@%@4320@%@132@%@5349@%@4117@%@82@%@207252@%@169996@%@2297@%@151418@%@96722@%@6465@%@128149@%@115193@%@9032@%@4535@%@4113@%@33@%@54154@%@45594@%@1280@%@13569@%@12099@%@353@%@21294@%@18997@%@720@%@9425@%@7148@%@145@%@8665@%@8266@%@218@%@8576@%@6892@%@338@%@8883@%@6273@%@245@%@15779@%@13766@%@764@%@28920@%@27181@%@643@%@14881@%@11728@%@491@%@24169@%@20874@%@824@%@10015@%@9261@%@193@%@55526@%@50876@%@833@%@9881@%@8128@%@250@%@12317@%@10668@%@632@%@31217@%@29332@%@753@%@3975@%@3330@%@112@%@6932@%@6158@%@105@%@7320@%@6696@%@125@%@16735@%@9010@%@175@%@5197@%@3587@%@101@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-26@%@0@%@0@%@0@%@1322866@%@1128672@%@35786@%@9531@%@7836@%@383@%@33825@%@30571@%@910@%@29395@%@21988@%@612@%@7256@%@6422@%@110@%@4914@%@4575@%@147@%@27425@%@21298@%@665@%@335900@%@321033@%@5408@%@4774@%@4336@%@133@%@5373@%@4169@%@82@%@209566@%@170864@%@2304@%@152071@%@97841@%@6595@%@128690@%@115728@%@9064@%@4574@%@4135@%@33@%@54639@%@46026@%@1287@%@13681@%@12132@%@354@%@21480@%@19106@%@722@%@9437@%@7264@%@146@%@8684@%@8280@%@218@%@8641@%@7013@%@338@%@8999@%@6346@%@249@%@15818@%@13810@%@765@%@29017@%@27328@%@643@%@14947@%@11849@%@493@%@24299@%@20990@%@829@%@10025@%@9283@%@193@%@55865@%@51082@%@837@%@9966@%@8159@%@254@%@12385@%@10783@%@636@%@31286@%@29417@%@757@%@3981@%@3379@%@112@%@6956@%@6179@%@105@%@7357@%@6739@%@125@%@16895@%@9047@%@175@%@5214@%@3664@%@102@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-27@%@0@%@0@%@0@%@1329074@%@1136054@%@35981@%@9534@%@7844@%@384@%@33976@%@30918@%@916@%@29430@%@22030@%@613@%@7300@%@6510@%@111@%@4927@%@4587@%@147@%@27653@%@21516@%@674@%@337637@%@322207@%@5453@%@4776@%@4349@%@135@%@5456@%@4237@%@82@%@210442@%@171515@%@2309@%@152701@%@100341@%@6661@%@129135@%@116245@%@9088@%@4601@%@4160@%@33@%@55017@%@46493@%@1294@%@13778@%@12225@%@356@%@21667@%@19231@%@726@%@9622@%@7333@%@146@%@8695@%@8287@%@218@%@8673@%@7041@%@338@%@9139@%@6415@%@256@%@15878@%@13927@%@766@%@29098@%@27419@%@643@%@14989@%@11875@%@492@%@24418@%@21108@%@832@%@10039@%@9283@%@193@%@56114@%@51320@%@840@%@10013@%@8222@%@257@%@12467@%@10818@%@637@%@31335@%@29473@%@761@%@3982@%@3379@%@113@%@6984@%@6193@%@105@%@7390@%@6768@%@125@%@16988@%@9078@%@175@%@5220@%@3707@%@102@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-02-28@%@0@%@0@%@0@%@1334634@%@1142703@%@36166@%@9541@%@7844@%@384@%@34215@%@31077@%@919@%@29476@%@22050@%@613@%@7347@%@6597@%@111@%@4933@%@4598@%@147@%@27823@%@21775@%@678@%@339735@%@322816@%@5495@%@4794@%@4359@%@135@%@5488@%@4246@%@82@%@211212@%@172049@%@2360@%@153028@%@103073@%@6675@%@129459@%@116708@%@9117@%@4640@%@4187@%@33@%@55391@%@46958@%@1299@%@13852@%@12289@%@358@%@21837@%@19346@%@728@%@9694@%@7350@%@148@%@8702@%@8291@%@219@%@8740@%@7111@%@341@%@9247@%@6441@%@257@%@15914@%@14080@%@772@%@29182@%@27470@%@644@%@15002@%@11893@%@492@%@24528@%@21214@%@837@%@10043@%@9283@%@194@%@56198@%@51564@%@840@%@10025@%@8227@%@259@%@12535@%@10913@%@644@%@31398@%@29564@%@763@%@3982@%@3379@%@113@%@6984@%@6193@%@105@%@7423@%@6796@%@126@%@17043@%@9127@%@175@%@5223@%@3835@%@103@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-01@%@0@%@0@%@0@%@1341314@%@1151915@%@36325@%@9543@%@7860@%@384@%@34380@%@31309@%@927@%@29497@%@22100@%@614@%@7374@%@6650@%@112@%@4936@%@4608@%@147@%@27967@%@22049@%@682@%@341793@%@326436@%@5540@%@4798@%@4379@%@136@%@5489@%@4294@%@83@%@212874@%@173741@%@2364@%@153685@%@103966@%@6689@%@129800@%@117165@%@9147@%@4640@%@4187@%@33@%@55828@%@47459@%@1309@%@13925@%@12345@%@362@%@22043@%@19474@%@732@%@9806@%@7403@%@148@%@8715@%@8311@%@220@%@8790@%@7134@%@342@%@9287@%@6634@%@262@%@15970@%@14185@%@776@%@29204@%@27500@%@647@%@15021@%@11958@%@495@%@24666@%@21314@%@838@%@10054@%@9285@%@194@%@56339@%@51788@%@843@%@10099@%@8261@%@259@%@12608@%@10983@%@650@%@31453@%@29639@%@765@%@3987@%@3391@%@113@%@6984@%@6193@%@105@%@7441@%@6807@%@126@%@17084@%@9168@%@177@%@5234@%@3939@%@104@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-02@%@0@%@0@%@0@%@1347026@%@1160863@%@36518@%@9557@%@7864@%@384@%@34547@%@31534@%@935@%@29594@%@22281@%@614@%@7400@%@6704@%@112@%@4949@%@4618@%@147@%@28117@%@22315@%@686@%@342371@%@328504@%@5583@%@4802@%@4395@%@136@%@5534@%@4307@%@84@%@214528@%@175980@%@2413@%@154669@%@104786@%@6696@%@130212@%@117693@%@9179@%@4662@%@4232@%@33@%@56138@%@48037@%@1321@%@14009@%@12464@%@364@%@22230@%@19580@%@733@%@9854@%@7465@%@149@%@8724@%@8323@%@220@%@8795@%@7148@%@342@%@9404@%@7176@%@265@%@16014@%@14223@%@776@%@29275@%@27587@%@650@%@15026@%@11976@%@495@%@24795@%@21422@%@841@%@10061@%@9302@%@197@%@56517@%@52094@%@853@%@10133@%@8336@%@260@%@12658@%@11057@%@658@%@31542@%@29745@%@766@%@3987@%@3403@%@113@%@7021@%@6223@%@105@%@7470@%@6835@%@126@%@17165@%@9216@%@178@%@5266@%@4038@%@104@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-03@%@0@%@0@%@0@%@1353834@%@1169916@%@36721@%@9570@%@7870@%@387@%@34745@%@31756@%@943@%@29666@%@22308@%@614@%@7461@%@6787@%@112@%@4982@%@4632@%@147@%@28346@%@22475@%@693@%@343807@%@330949@%@5624@%@4819@%@4401@%@136@%@5543@%@4365@%@84@%@216423@%@178213@%@2462@%@155217@%@106110@%@6718@%@130642@%@118207@%@9212@%@4696@%@4257@%@33@%@56487@%@48496@%@1328@%@14103@%@12494@%@365@%@22409@%@19653@%@736@%@9882@%@7632@%@150@%@8730@%@8342@%@220@%@8804@%@7166@%@342@%@9545@%@7302@%@267@%@16064@%@14284@%@779@%@29380@%@27665@%@651@%@15047@%@11995@%@495@%@24929@%@21543@%@845@%@10080@%@9332@%@197@%@56780@%@52364@%@860@%@10199@%@8424@%@261@%@12717@%@11126@%@664@%@31621@%@29910@%@769@%@4030@%@3414@%@113@%@7044@%@6239@%@105@%@7517@%@6864@%@126@%@17281@%@9252@%@178@%@5268@%@4089@%@105@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-04@%@0@%@0@%@0@%@1361098@%@1176356@%@36897@%@9580@%@7872@%@387@%@35013@%@32012@%@960@%@29677@%@22324@%@614@%@7493@%@6823@%@112@%@4994@%@4642@%@147@%@28574@%@22755@%@698@%@345816@%@332682@%@5667@%@4842@%@4408@%@136@%@5587@%@4386@%@84@%@218154@%@179430@%@2507@%@155808@%@106818@%@6731@%@131046@%@118705@%@9238@%@4737@%@4278@%@33@%@56999@%@48908@%@1333@%@14171@%@12564@%@365@%@22576@%@19743@%@740@%@9947@%@7703@%@150@%@8742@%@8359@%@220@%@8810@%@7169@%@343@%@9648@%@7348@%@269@%@16107@%@14330@%@780@%@29467@%@27768@%@652@%@15065@%@12006@%@495@%@25065@%@21659@%@847@%@10083@%@9338@%@198@%@56976@%@52678@%@862@%@10276@%@8477@%@262@%@12773@%@11185@%@666@%@31708@%@29980@%@773@%@4036@%@3423@%@113@%@7069@%@6250@%@105@%@7544@%@6901@%@126@%@17444@%@9300@%@178@%@5271@%@4132@%@106@%@0@%@0@%@0@)']
sdrin=['https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-05@%@0@%@0@%@0@%@1368069@%@1182687@%@37026@%@9589@%@7876@%@387@%@35227@%@32264@%@966@%@30192@%@22444@%@614@%@7543@%@6904@%@112@%@5009@%@4652@%@147@%@28749@%@22850@%@698@%@346975@%@333848@%@5712@%@4851@%@4425@%@136@%@5594@%@4418@%@85@%@219940@%@181128@%@2514@%@156757@%@107109@%@6742@%@131450@%@119181@%@9257@%@4766@%@4308@%@33@%@57292@%@49422@%@1340@%@14323@%@12646@%@366@%@22773@%@19966@%@744@%@9977@%@7767@%@152@%@8761@%@8376@%@220@%@8851@%@7170@%@344@%@9798@%@7588@%@272@%@16170@%@14376@%@781@%@29554@%@27894@%@653@%@15076@%@12022@%@496@%@25164@%@21787@%@851@%@10091@%@9361@%@200@%@57172@%@52941@%@869@%@10289@%@8487@%@263@%@12818@%@11264@%@670@%@31837@%@30079@%@774@%@4040@%@3436@%@113@%@7078@%@6266@%@105@%@7571@%@6915@%@126@%@17507@%@9333@%@178@%@5285@%@4184@%@106@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-06@%@0@%@0@%@0@%@1373836@%@1189510@%@37154@%@9605@%@7894@%@387@%@35380@%@32483@%@971@%@30205@%@23163@%@627@%@7618@%@6989@%@113@%@5017@%@4662@%@147@%@28838@%@23099@%@703@%@348591@%@335545@%@5756@%@4864@%@4435@%@137@%@5611@%@4452@%@85@%@221034@%@182453@%@2515@%@157343@%@107156@%@6743@%@131843@%@119644@%@9277@%@4839@%@4334@%@33@%@57688@%@49896@%@1344@%@14445@%@12810@%@368@%@22944@%@20147@%@748@%@10038@%@7802@%@152@%@8774@%@8394@%@220@%@8854@%@7196@%@345@%@9928@%@7618@%@280@%@16228@%@14474@%@782@%@29646@%@27982@%@654@%@15103@%@12030@%@498@%@25251@%@21877@%@854@%@10093@%@9385@%@200@%@57337@%@53251@%@873@%@10323@%@8491@%@263@%@12859@%@11333@%@670@%@31905@%@30166@%@778@%@4052@%@3455@%@113@%@7116@%@6319@%@105@%@7615@%@6944@%@126@%@17560@%@9360@%@179@%@5289@%@4271@%@108@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-07@%@0@%@0@%@0@%@1379662@%@1194656@%@37266@%@9609@%@7894@%@387@%@35527@%@32699@%@973@%@30217@%@23174@%@628@%@7655@%@7045@%@116@%@5027@%@4672@%@149@%@28987@%@23204@%@707@%@350425@%@337371@%@5801@%@4866@%@4435@%@138@%@5631@%@4479@%@86@%@222400@%@182991@%@2517@%@157738@%@107731@%@6744@%@132171@%@120074@%@9295@%@4906@%@4354@%@33@%@58039@%@50298@%@1353@%@14563@%@12854@%@370@%@23140@%@20252@%@750@%@10067@%@7813@%@154@%@8787@%@8411@%@220@%@8870@%@7197@%@345@%@9971@%@7673@%@282@%@16293@%@14510@%@787@%@29706@%@28022@%@654@%@15135@%@12031@%@498@%@25329@%@21965@%@856@%@10098@%@9385@%@200@%@57527@%@53403@%@874@%@10361@%@8507@%@264@%@12907@%@11384@%@670@%@32039@%@30266@%@781@%@4062@%@3455@%@113@%@7118@%@6420@%@105@%@7630@%@6987@%@126@%@17565@%@9398@%@181@%@5296@%@4302@%@109@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-08@%@0@%@0@%@0@%@1386556@%@1203381@%@37547@%@9616@%@7898@%@387@%@35727@%@32934@%@978@%@30808@%@23516@%@628@%@7717@%@7094@%@117@%@5067@%@4683@%@150@%@29154@%@23365@%@713@%@352208@%@338855@%@5849@%@4866@%@4439@%@138@%@5637@%@4526@%@86@%@224139@%@184900@%@2589@%@158609@%@110036@%@6824@%@132499@%@120498@%@9326@%@4906@%@4354@%@33@%@58181@%@50692@%@1359@%@14633@%@12904@%@374@%@23300@%@20471@%@754@%@10107@%@7855@%@155@%@8804@%@8414@%@221@%@8872@%@7198@%@346@%@10105@%@7795@%@286@%@16358@%@14588@%@788@%@29714@%@28046@%@655@%@15136@%@12078@%@498@%@25423@%@22062@%@859@%@10107@%@9410@%@201@%@57606@%@53591@%@877@%@10387@%@8643@%@264@%@12960@%@11463@%@672@%@32103@%@30337@%@783@%@4079@%@3455@%@114@%@7133@%@6516@%@106@%@7642@%@7001@%@127@%@17654@%@9438@%@181@%@5299@%@4326@%@109@%@0@%@0@%@0@)','https://sehatdirumah.com/input?key=9zQVh9tS3BJcaWNtAqL5&data=(@2021-03-09@%@0@%@0@%@0@%@1392945@%@1210877@%@37757@%@9642@%@7924@%@387@%@35983@%@33216@%@985@%@30908@%@23760@%@629@%@7766@%@7151@%@118@%@5075@%@4693@%@150@%@29330@%@23555@%@715@%@353075@%@340510@%@5894@%@4866@%@4460@%@138@%@5657@%@4562@%@87@%@225926@%@185877@%@2639@%@159507@%@111907@%@6868@%@132890@%@120926@%@9356@%@4956@%@4398@%@33@%@58441@%@51128@%@1365@%@14766@%@13029@%@374@%@23426@%@20545@%@758@%@10182@%@7991@%@155@%@8807@%@8436@%@221@%@8883@%@7203@%@346@%@10241@%@7885@%@288@%@16390@%@14636@%@791@%@29797@%@28116@%@658@%@15146@%@12121@%@498@%@25521@%@22153@%@865@%@10120@%@9421@%@201@%@57698@%@53729@%@878@%@10442@%@8765@%@264@%@13020@%@11528@%@672@%@32212@%@30412@%@786@%@4112@%@3467@%@114@%@7142@%@6526@%@106@%@7699@%@7044@%@127@%@18014@%@9467@%@182@%@5305@%@4336@%@109@%@0@%@0@%@0@)']
for sd in sdrin:
    r=requests.get(sd)
    print(r.content)

b'data berhasil dimasukkan'
b'data berhasil dimasukkan'
b'data berhasil dimasukkan'
b'data berhasil dimasukkan'
b'data berhasil dimasukkan'
